# 역전파 알고리즘 구현 - 실습 노트북 (중급 단계)

이 노트북은 역전파(Backpropagation) 알고리즘을 처음부터 구현하고 검증합니다.

## 목차
1. 역전파 알고리즘 구현
2. 경사하강법 구현
3. 완전한 학습 예제
4. 그래디언트 검증 (Gradient Checking)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 시드 설정 (재현성)
np.random.seed(42)

print("NumPy version:", np.__version__)

## 1. 역전파 알고리즘 구현

### 핵심 수식

**출력층 그래디언트:**
$$\delta^{[L]} = \frac{\partial L}{\partial z^{[L]}}$$

**은닉층 그래디언트:**
$$\delta^{[l]} = (W^{[l+1]})^T \delta^{[l+1]} \odot f'(z^{[l]})$$

**파라미터 그래디언트:**
$$\frac{\partial L}{\partial W^{[l]}} = \delta^{[l]} (a^{[l-1]})^T$$
$$\frac{\partial L}{\partial b^{[l]}} = \delta^{[l]}$$

In [ ]:
# 활성화 함수와 미분
def sigmoid(z):
    return 1 / (1 + np.exp(-np.clip(z, -500, 500)))  # 오버플로우 방지

def sigmoid_derivative(z):
    s = sigmoid(z)
    return s * (1 - s)

def relu(z):
    return np.maximum(0, z)

def relu_derivative(z):
    return np.where(z > 0, 1, 0)

# 손실 함수
def binary_cross_entropy(y_true, y_pred):
    """이진 교차 엔트로피 손실"""
    epsilon = 1e-10
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

def binary_cross_entropy_derivative(y_true, y_pred):
    """이진 교차 엔트로피 손실의 미분 (sigmoid 활성화 함수와 결합시)"""
    return y_pred - y_true

In [ ]:
def forward_propagation(X, W1, b1, W2, b2):
    """
    2층 신경망의 순방향 전파
    
    Parameters:
    -----------
    X : (n_features, n_samples) 입력 데이터
    W1 : (n_hidden, n_features) 1층 가중치
    b1 : (n_hidden, 1) 1층 편향
    W2 : (n_output, n_hidden) 2층 가중치
    b2 : (n_output, 1) 2층 편향
    
    Returns:
    --------
    cache : dict, 중간 계산 값들
    """
    # 1층 (은닉층, ReLU)
    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    
    # 2층 (출력층, Sigmoid)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    
    cache = {
        'X': X,
        'Z1': Z1,
        'A1': A1,
        'Z2': Z2,
        'A2': A2
    }
    
    return A2, cache

def backward_propagation(Y, cache, W1, W2):
    """
    2층 신경망의 역전파
    
    Parameters:
    -----------
    Y : (1, n_samples) 실제 레이블
    cache : dict, 순방향 전파의 중간 값들
    W1, W2 : 가중치 행렬
    
    Returns:
    --------
    grads : dict, 그래디언트들
    """
    m = Y.shape[1]  # 샘플 개수
    
    X = cache['X']
    A1 = cache['A1']
    A2 = cache['A2']
    Z1 = cache['Z1']
    
    # 출력층 그래디언트 (Sigmoid + Cross-Entropy)
    dZ2 = A2 - Y
    
    # 출력층 파라미터 그래디언트
    dW2 = (1/m) * np.dot(dZ2, A1.T)
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)
    
    # 은닉층 그래디언트
    dZ1 = np.dot(W2.T, dZ2) * relu_derivative(Z1)
    
    # 은닉층 파라미터 그래디언트
    dW1 = (1/m) * np.dot(dZ1, X.T)
    db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)
    
    grads = {
        'dW1': dW1,
        'db1': db1,
        'dW2': dW2,
        'db2': db2
    }
    
    return grads

### 수치 예제 검증

문서의 예제와 동일한 값으로 역전파를 검증합니다.

In [ ]:
# 문서의 예제와 동일한 설정
X = np.array([[1.0], [2.0]])
Y = np.array([[1.0]])

W1 = np.array([[0.5, 0.3],
               [0.2, 0.4]])
b1 = np.array([[0.1], [0.2]])

W2 = np.array([[0.6, 0.7]])
b2 = np.array([[0.15]])

# 순방향 전파
A2, cache = forward_propagation(X, W1, b1, W2, b2)

print("=" * 70)
print("순방향 전파 결과")
print("=" * 70)
print(f"Z1 = {cache['Z1'].flatten()}")
print(f"A1 = {cache['A1'].flatten()}")
print(f"Z2 = {cache['Z2'].flatten()}")
print(f"A2 (예측) = {A2.flatten()}")
print()

# 손실 계산
loss = binary_cross_entropy(Y, A2)
print(f"손실 = {loss.flatten()[0]:.4f}")
print("=" * 70)

# 역전파
grads = backward_propagation(Y, cache, W1, W2)

print("\n" + "=" * 70)
print("역전파 결과 (그래디언트)")
print("=" * 70)
print("출력층:")
print(f"  dW2 = {grads['dW2']}")
print(f"  db2 = {grads['db2'].flatten()}")
print()
print("은닉층:")
print(f"  dW1 = \n{grads['dW1']}")
print(f"  db1 = {grads['db1'].flatten()}")
print("=" * 70)

## 2. 경사하강법 구현

### 수식
$$W := W - \alpha \frac{\partial L}{\partial W}$$
$$b := b - \alpha \frac{\partial L}{\partial b}$$

In [ ]:
def update_parameters(W1, b1, W2, b2, grads, learning_rate):
    """
    경사하강법으로 파라미터 업데이트
    
    Parameters:
    -----------
    W1, b1, W2, b2 : 현재 파라미터
    grads : dict, 그래디언트
    learning_rate : float, 학습률
    
    Returns:
    --------
    W1, b1, W2, b2 : 업데이트된 파라미터
    """
    W1 = W1 - learning_rate * grads['dW1']
    b1 = b1 - learning_rate * grads['db1']
    W2 = W2 - learning_rate * grads['dW2']
    b2 = b2 - learning_rate * grads['db2']
    
    return W1, b1, W2, b2

# 파라미터 업데이트 예제
learning_rate = 0.1

print("=" * 70)
print("파라미터 업데이트 (학습률 = 0.1)")
print("=" * 70)
print("업데이트 전:")
print(f"  W2 = {W2}")
print(f"  b2 = {b2.flatten()}")

W1_new, b1_new, W2_new, b2_new = update_parameters(
    W1.copy(), b1.copy(), W2.copy(), b2.copy(), grads, learning_rate
)

print("\n업데이트 후:")
print(f"  W2 = {W2_new}")
print(f"  b2 = {b2_new.flatten()}")

print("\n수동 계산 (W2):")
print(f"  W2_new = {W2} - {learning_rate} × {grads['dW2']}")
print(f"         = {W2 - learning_rate * grads['dW2']}")
print("=" * 70)

## 3. 완전한 학습 예제

단순 선형 분류 문제를 학습합니다.

In [ ]:
# 데이터 생성
np.random.seed(42)

# 클래스 0: 원점 주변
X0 = np.random.randn(2, 50) * 0.5 - 1
Y0 = np.zeros((1, 50))

# 클래스 1: (2, 2) 주변
X1 = np.random.randn(2, 50) * 0.5 + 1
Y1 = np.ones((1, 50))

# 데이터 결합
X_train = np.hstack([X0, X1])
Y_train = np.hstack([Y0, Y1])

# 데이터 섞기
shuffle_idx = np.random.permutation(X_train.shape[1])
X_train = X_train[:, shuffle_idx]
Y_train = Y_train[:, shuffle_idx]

print(f"훈련 데이터 크기: {X_train.shape}")
print(f"레이블 크기: {Y_train.shape}")

# 데이터 시각화
plt.figure(figsize=(8, 6))
plt.scatter(X_train[0, Y_train[0]==0], X_train[1, Y_train[0]==0], 
            c='blue', label='Class 0', alpha=0.6)
plt.scatter(X_train[0, Y_train[0]==1], X_train[1, Y_train[0]==1], 
            c='red', label='Class 1', alpha=0.6)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Training Data')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
def train_neural_network(X, Y, n_hidden=4, learning_rate=0.1, num_iterations=1000):
    """
    신경망 학습
    
    Parameters:
    -----------
    X : (n_features, n_samples)
    Y : (1, n_samples)
    n_hidden : int, 은닉층 뉴런 개수
    learning_rate : float
    num_iterations : int
    
    Returns:
    --------
    parameters : dict
    losses : list
    """
    np.random.seed(42)
    n_features = X.shape[0]
    
    # 파라미터 초기화 (Xavier initialization)
    W1 = np.random.randn(n_hidden, n_features) * np.sqrt(2.0 / n_features)
    b1 = np.zeros((n_hidden, 1))
    W2 = np.random.randn(1, n_hidden) * np.sqrt(2.0 / n_hidden)
    b2 = np.zeros((1, 1))
    
    losses = []
    
    for i in range(num_iterations):
        # 순방향 전파
        A2, cache = forward_propagation(X, W1, b1, W2, b2)
        
        # 손실 계산
        loss = np.mean(binary_cross_entropy(Y, A2))
        losses.append(loss)
        
        # 역전파
        grads = backward_propagation(Y, cache, W1, W2)
        
        # 파라미터 업데이트
        W1, b1, W2, b2 = update_parameters(W1, b1, W2, b2, grads, learning_rate)
        
        # 진행 상황 출력
        if i % 100 == 0:
            # 정확도 계산
            predictions = (A2 > 0.5).astype(float)
            accuracy = np.mean(predictions == Y) * 100
            print(f"반복 {i:4d}: 손실 = {loss:.4f}, 정확도 = {accuracy:.2f}%")
    
    parameters = {
        'W1': W1, 'b1': b1,
        'W2': W2, 'b2': b2
    }
    
    return parameters, losses

# 학습 실행
print("=" * 70)
print("신경망 학습 시작")
print("=" * 70)
parameters, losses = train_neural_network(X_train, Y_train, 
                                         n_hidden=4, 
                                         learning_rate=0.5, 
                                         num_iterations=1000)
print("=" * 70)

In [ ]:
# 학습 곡선 시각화
plt.figure(figsize=(10, 5))
plt.plot(losses, linewidth=2)
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Loss (Cross-Entropy)', fontsize=12)
plt.title('Training Loss Over Time', fontsize=14)
plt.grid(True, alpha=0.3)
plt.show()

print(f"\n초기 손실: {losses[0]:.4f}")
print(f"최종 손실: {losses[-1]:.4f}")
print(f"손실 감소: {(losses[0] - losses[-1]) / losses[0] * 100:.2f}%")

In [ ]:
# 결정 경계 시각화
def plot_decision_boundary(X, Y, parameters):
    """결정 경계 시각화"""
    # 그리드 생성
    x_min, x_max = X[0, :].min() - 1, X[0, :].max() + 1
    y_min, y_max = X[1, :].min() - 1, X[1, :].max() + 1
    
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                         np.linspace(y_min, y_max, 200))
    
    # 그리드 포인트 예측
    X_grid = np.c_[xx.ravel(), yy.ravel()].T
    A2, _ = forward_propagation(X_grid, 
                                parameters['W1'], parameters['b1'],
                                parameters['W2'], parameters['b2'])
    Z = A2.reshape(xx.shape)
    
    # 플롯
    plt.figure(figsize=(10, 8))
    plt.contourf(xx, yy, Z, levels=20, cmap='RdBu', alpha=0.6)
    plt.colorbar(label='Probability of Class 1')
    
    # 데이터 포인트
    plt.scatter(X[0, Y[0]==0], X[1, Y[0]==0], 
               c='blue', edgecolors='k', s=80, label='Class 0')
    plt.scatter(X[0, Y[0]==1], X[1, Y[0]==1], 
               c='red', edgecolors='k', s=80, label='Class 1')
    
    plt.xlabel('Feature 1', fontsize=12)
    plt.ylabel('Feature 2', fontsize=12)
    plt.title('Decision Boundary', fontsize=14)
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.show()

plot_decision_boundary(X_train, Y_train, parameters)

## 4. 그래디언트 검증 (Gradient Checking)

수치적 그래디언트와 역전파 그래디언트를 비교하여 구현이 올바른지 검증합니다.

### 수식
수치적 그래디언트:
$$\frac{\partial L}{\partial \theta} \approx \frac{L(\theta + \epsilon) - L(\theta - \epsilon)}{2\epsilon}$$

In [ ]:
def gradient_check(X, Y, parameters, grads, epsilon=1e-7):
    """
    그래디언트 검증
    
    Parameters:
    -----------
    X, Y : 입력 데이터와 레이블
    parameters : dict, 파라미터
    grads : dict, 역전파로 계산된 그래디언트
    epsilon : float, 수치 미분을 위한 작은 값
    
    Returns:
    --------
    difference : float, 상대 오차
    """
    # 파라미터를 벡터로 변환
    params_values = []
    grads_values = []
    
    for key in ['W1', 'b1', 'W2', 'b2']:
        params_values.append(parameters[key].flatten())
        grads_values.append(grads['d' + key].flatten())
    
    theta = np.concatenate(params_values)
    grad_bp = np.concatenate(grads_values)
    
    # 수치적 그래디언트 계산
    num_grad = np.zeros_like(theta)
    
    for i in range(len(theta)):
        # theta + epsilon
        theta_plus = theta.copy()
        theta_plus[i] += epsilon
        
        # 파라미터 복원
        idx = 0
        params_plus = {}
        for key in ['W1', 'b1', 'W2', 'b2']:
            shape = parameters[key].shape
            size = np.prod(shape)
            params_plus[key] = theta_plus[idx:idx+size].reshape(shape)
            idx += size
        
        # 손실 계산
        A2_plus, _ = forward_propagation(X, params_plus['W1'], params_plus['b1'],
                                        params_plus['W2'], params_plus['b2'])
        loss_plus = np.mean(binary_cross_entropy(Y, A2_plus))
        
        # theta - epsilon
        theta_minus = theta.copy()
        theta_minus[i] -= epsilon
        
        idx = 0
        params_minus = {}
        for key in ['W1', 'b1', 'W2', 'b2']:
            shape = parameters[key].shape
            size = np.prod(shape)
            params_minus[key] = theta_minus[idx:idx+size].reshape(shape)
            idx += size
        
        A2_minus, _ = forward_propagation(X, params_minus['W1'], params_minus['b1'],
                                         params_minus['W2'], params_minus['b2'])
        loss_minus = np.mean(binary_cross_entropy(Y, A2_minus))
        
        # 수치적 그래디언트
        num_grad[i] = (loss_plus - loss_minus) / (2 * epsilon)
    
    # 상대 오차 계산
    numerator = np.linalg.norm(grad_bp - num_grad)
    denominator = np.linalg.norm(grad_bp) + np.linalg.norm(num_grad)
    difference = numerator / denominator
    
    return difference, grad_bp, num_grad

# 작은 데이터로 테스트
X_test = X_train[:, :5]
Y_test = Y_train[:, :5]

# 순방향 전파
A2, cache = forward_propagation(X_test, 
                                parameters['W1'], parameters['b1'],
                                parameters['W2'], parameters['b2'])

# 역전파
grads = backward_propagation(Y_test, cache, 
                            parameters['W1'], parameters['W2'])

print("=" * 70)
print("그래디언트 검증 (샘플 몇 개만 사용)")
print("=" * 70)
print("계산 중... (시간이 걸릴 수 있습니다)")

difference, grad_bp, num_grad = gradient_check(X_test, Y_test, parameters, grads)

print(f"\n상대 오차: {difference:.10f}")
print()
if difference < 1e-7:
    print("✓ 역전파 구현이 정확합니다!")
elif difference < 1e-5:
    print("⚠ 역전파가 대체로 정확하지만 확인이 필요할 수 있습니다.")
else:
    print("✗ 역전파 구현에 문제가 있을 수 있습니다.")

print("\n그래디언트 비교 (처음 5개 값):")
print(f"  역전파:     {grad_bp[:5]}")
print(f"  수치적:     {num_grad[:5]}")
print(f"  차이:       {grad_bp[:5] - num_grad[:5]}")
print("=" * 70)

## 요약

이 노트북에서 다룬 내용:

1. **역전파 알고리즘**: 처음부터 구현하고 수치적으로 검증
2. **경사하강법**: 파라미터 업데이트 메커니즘
3. **완전한 학습**: 실제 데이터로 신경망 학습
4. **그래디언트 검증**: 수치적 방법으로 구현 정확도 확인

### 주요 결과
- 역전파 구현이 정확함을 수치적으로 검증
- 신경망이 데이터를 성공적으로 학습
- 결정 경계를 시각화하여 학습 결과 확인

### 다음 단계
다음 노트북에서는 **Adam, RMSprop** 등의 고급 최적화 기법과 **정규화** 기법을 구현합니다.